In [31]:
import pymysql
import pandas as pd
import numpy as np

In [32]:
def easy_ddl(connection, query):
    try:
        with connection.cursor() as cursor:
            cursor.execute(query)
            connection.commit()
        print("Successfully Executed")
    except pymysql.MySQLError as e:
        print(f"Error: {e}")

def easy_query(connection, query):
    try:
        with connection.cursor() as cursor:
            cursor.execute(query)
            results = cursor.fetchall()
            print("Successfully Executed")
            return pd.DataFrame(results)
    except pymysql.MySQLError as e:
        print(f"Error: {e}")


<img src= "https://upload.wikimedia.org/wikipedia/labs/8/8e/Mysql_logo.png" style="height:150px; width:300px; "> 
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/93/Amazon_Web_Services_Logo.svg/2560px-Amazon_Web_Services_Logo.svg.png" style="height:150px; width:300px; padding:0px 100px" >

</br>
</br>
</br>

# AWS MySQL setup

This is to setup my AWS RDS MySQL Database to track my progress on LeetCodee SQL questions. </br>
You can connect this to a local database - otherwise, you can set up your cloud database <a href="https://aws.amazon.com/rds/">here.</a> (or your cloud database of choice.)

## Create Connection

In [33]:
endpoint = ""
port = 3306
username = ""
password = ""

try:
    aws_connection = pymysql.connect(
        host=endpoint,
        user=username,
        password=password,
        #database=database,
        port=port
    )
    print("Connection successful!")

    aws_cursor = aws_connection.cursor()
    
except pymysql.MySQLError as e:
    print(f"Error: {e}")

Connection successful!


## Creating the AWS Database and Table

In [34]:
## Creating the database
easy_ddl(aws_connection, "CREATE DATABASE IF NOT EXISTS leetcode;")

## Use the database
easy_ddl(aws_connection, "USE leetcode;")

## Creating the table
query = """CREATE TABLE IF NOT EXISTS daily_problems (
    id INT PRIMARY KEY,
    date DATETIME DEFAULT CURRENT_TIMESTAMP,  -- Use CURRENT_TIMESTAMP for DATETIME
    name VARCHAR(225),
    complexity VARCHAR(8) CHECK (complexity IN ('Easy', 'Medium', 'Hard')),
    acceptance_rate FLOAT,
    time FLOAT,
    language VARCHAR(20) DEFAULT 'Python',
    chat_gpt INT CHECK (chat_gpt IN (0, 1)),
    speed FLOAT,
    memory FLOAT,
    skills VARCHAR(225),
    notes VARCHAR(225));
    """
    
easy_ddl(aws_connection, query)

Successfully Executed
Successfully Executed
Successfully Executed
Successfully Executed


## Move Data From Local Database
I had already started storing my data from a local database. Here I will beging moving data into the AWS database.

In [35]:
endpoint = "localhost"
port = 3306
username = "root"
password = ""
database = "leetcode"

try:
    local_connection = pymysql.connect(
        host=endpoint,
        user=username,
        password=password,
        database=database,
        port=port
    )
    print("Connection successful!")
    
except pymysql.MySQLError as e:
    print(f"Error: {e}")

Connection successful!


In [36]:
df = easy_query(local_connection, "SELECT * FROM daily_problems")
col_names = easy_query(local_connection, "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'daily_problems'")[0]

Successfully Executed
Successfully Executed


In [37]:
df.columns = col_names
df.head(3)

,problem_id,prob_date,prob_name,accpetance,complexity,completion_time,skills,notes,chat_gpt,accepted,speed,memory
0,725,2024-09-08 07:38:16,Split Linked List in Parts,67.5,Medium,1186.32,Linked List,Very slow solution,0.0,1.0,NaN,NaN
1,874,2024-09-04 11:10:35,Walking Robot Simulation,52.2,Medium,4326.63,"Array, Hash Table, Simulation",I completed 3 solution but none of them was fa...,0.0,1.0,NaN,NaN
2,947,2024-08-29 20:19:00,Most Stones Removed with Same Row or Column,62.0,Medium,3104.14,"Hash Table, Depth-First Search, Union Find, Graph",I'm so tired. I tried a solution with an adjac...,NaN,NaN,NaN,NaN


In [38]:
# Close local connection
local_connection.close()

### Transfering data
Sending the data to AWS

In [39]:
with aws_connection.cursor() as cursor:
    query = """
    INSERT INTO daily_problems (id, date, name, complexity, acceptance_rate, time, chat_gpt, speed, memory, skills, notes)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    try:
        for index, row in df.iterrows():
            # Replace NaN values with None
            row = row.replace({np.nan: None})

            try:
                # Execute the query for each row
                cursor.execute(query, (
                    row['problem_id'], 
                    row['prob_date'], 
                    row['prob_name'], 
                    row['complexity'], 
                    row['accpetance'],
                    row['completion_time'], 
                    row['chat_gpt'], 
                    row['speed'], 
                    row['memory'], 
                    row['skills'],
                    row['notes']
                ))
            except pymysql.MySQLError as e:
                #print(f"Error: {e}")
                continue
        
        aws_connection.commit()
        print("Successfully inserted rows into daily_problems")
    
    except pymysql.MySQLError as e:
        print(f"Error: {e}")

Successfully inserted rows into daily_problems


### Double Checking
Ensuring data was properly sent to AWS

In [40]:
aws_df = easy_query(aws_connection, "SELECT * FROM leetcode.daily_problems ORDER BY date DESC")
col_names = easy_query(aws_connection, "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'daily_problems'")[0]

aws_df.columns = col_names
aws_df.head(5)

Successfully Executed
Successfully Executed


,id,date,name,complexity,acceptance_rate,time,language,chat_gpt,speed,memory,skills,notes
0,2220,2024-09-11 10:00:22,Minimum Bit Flips to Convert Number,Easy,87.2,1.7063,Python,0.0,97.25,26.96,Bit Manipulation,
1,2807,2024-09-10 07:49:21,Insert Greatest Common Divisors in Linked List,Medium,91.7,2392.1100,Python,0.0,NaN,NaN,"LinkedList, Math, Number Theory",Tried to modify LinkedList in place. Ended up ...
2,2326,2024-09-09 07:19:57,Spiral Matrix IV,Medium,80.3,2206.8500,Python,0.0,NaN,NaN,"Array, LinkedList, Matrix, Simulation",Extremely slow and memory intensive
3,725,2024-09-08 07:38:16,Split Linked List in Parts,Medium,67.5,1186.3200,Python,0.0,NaN,NaN,Linked List,Very slow solution
4,1367,2024-09-07 11:29:00,Linked List in Binary Tree,Medium,48.5,3266.2200,Python,0.0,NaN,NaN,"Linked List, Tree, Depth-First Search, Breadth...","Had some interruptions, ended up using ChatGPT"


In [41]:
aws_connection.close()

# Next Steps:
Use <code>lcdc.py</code> to track and load your daily challenge data. You need Selenium and BeautifulSoup4.